In [4]:
import pandas as pd
import numpy as np
import glob
import os

# --- CONFIGURATION DES CHEMINS (À ADAPTER SELON VOTRE DOSSIER) ---
# Basé sur votre structure de fichiers fournie
PATH_FLIGHTLIST = "data/raw/flightlist_train.parquet" 
PATH_FUEL_TRAIN = "data/raw/fuel_train.parquet"
PATH_PRC_DIR = "data/raw/flights_train" # Répertoire contenant les fichiers prc*.parquet

def audit_data_quality():
    print(">>> DÉMARRAGE DE L'AUDIT DE QUALITÉ AÉRONAUTIQUE <<<\n")

    # 1. Chargement de la Référence (Flightlist)
    try:
        print(f"[1/3] Chargement de la Flightlist ({PATH_FLIGHTLIST})...")
        df_fl = pd.read_parquet(PATH_FLIGHTLIST)
        valid_ids = set(df_fl['flight_id'].unique())
        print(f"   -> {len(valid_ids)} vols référencés valides.")
    except Exception as e:
        print(f"   ERREUR CRITIQUE: Impossible de lire flightlist. {e}")
        return

    # 2. Analyse de la Cible (Fuel Train)
    try:
        print(f"\n[2/3] Analyse de la cible Fuel ({PATH_FUEL_TRAIN})...")
        df_fuel = pd.read_parquet(PATH_FUEL_TRAIN)
        
        # A. Doublons
        duplicates = df_fuel.duplicated().sum()
        
        # B. Quantification (Valeurs rondes)
        # On regarde le pourcentage de valeurs divisibles par 10 et 100
        fuel_vals = df_fuel['fuel_kg'].dropna()
        pct_mod_10 = (fuel_vals % 10 == 0).mean() * 100
        pct_mod_100 = (fuel_vals % 100 == 0).mean() * 100
        
        # C. Valeurs Aberrantes (Consommation négative ou nulle)
        zeros = (fuel_vals <= 0).sum()

        print(f"   -> Lignes totales : {len(df_fuel)}")
        print(f"   -> Doublons exacts : {duplicates} (Doivent être supprimés)")
        print(f"   -> Segments avec fuel <= 0 : {zeros}")
        print(f"   -> Hypothèse Capteurs : {pct_mod_10:.1f}% sont multiples de 10kg")
        print(f"   -> Hypothèse 'Block Fuel' : {pct_mod_100:.1f}% sont multiples de 100kg")
        
    except Exception as e:
        print(f"   ERREUR: Impossible de lire fuel_train. {e}")

    # 3. Analyse Physique des PRC
    print(f"\n[3/3] Analyse Physique des fichiers PRC...")
    prc_files = glob.glob(os.path.join(PATH_PRC_DIR, "prc*.parquet"))
    
    if not prc_files:
        print("   ATTENTION: Aucun fichier prc*.parquet trouvé dans le dossier.")
        return

    global_stats = {
        'total_points': 0,
        'ghost_flights_ids': set(), # ID présents dans PRC mais pas dans Flightlist
        'bad_altitude_count': 0,    # < -1000 ft ou > 60000 ft
        'bad_speed_count': 0,       # > 1200 kts (Mach 2 approx au sol)
        'nan_coords': 0             # Lat/Lon manquants
    }

    for f in prc_files:
        print(f"   -> Scan de {os.path.basename(f)}...")
        try:
            df = pd.read_parquet(f)
            global_stats['total_points'] += len(df)
            
            # Vérification Vols Fantômes
            current_ids = set(df['flight_id'].unique())
            ghosts = current_ids - valid_ids
            global_stats['ghost_flights_ids'].update(ghosts)
            
            # Vérification Physique (Vectorisée)
            # Altitude: Tolérance -1000ft (QNH erreur) à 60000ft (Plafond max civil)
            bad_alt = (df['altitude'] < -1000) | (df['altitude'] > 60000)
            global_stats['bad_altitude_count'] += bad_alt.sum()
            
            # Vitesse: > 1200 kts est physiquement impossible pour ces avions
            bad_spd = (df['groundspeed'] > 1200) | (df['groundspeed'] < 0)
            global_stats['bad_speed_count'] += bad_spd.sum()
            
            # Coordonnées
            global_stats['nan_coords'] += df[['latitude', 'longitude']].isna().any(axis=1).sum()
            
        except Exception as e:
            print(f"      Erreur lecture fichier: {e}")

    # --- RAPPORT FINAL ---
    print("\n>>> RAPPORT D'AUDIT <<<")
    print(f"Total points analysés : {global_stats['total_points']:,}")
    print(f"1. COHÉRENCE : {len(global_stats['ghost_flights_ids'])} vols fantômes détectés (à filtrer).")
    print(f"2. PHYSIQUE : {global_stats['bad_altitude_count']} points d'altitude aberrants.")
    print(f"3. PHYSIQUE : {global_stats['bad_speed_count']} points de vitesse aberrants.")
    print(f"4. QUALITÉ : {global_stats['nan_coords']} points sans coordonnées GPS.")
    
    if len(global_stats['ghost_flights_ids']) > 0:
        print("\nRECOMMANDATION EXPERT : Filtrez impérativement les vols fantômes avant le Feature Engineering.")
    if global_stats['bad_altitude_count'] > 0:
        print("RECOMMANDATION EXPERT : Appliquez un 'clip' ou supprimez les altitudes < -1000 ou > 60000.")

# Exécuter l'audit
audit_data_quality()

>>> DÉMARRAGE DE L'AUDIT DE QUALITÉ AÉRONAUTIQUE <<<

[1/3] Chargement de la Flightlist (data/raw/flightlist_train.parquet)...
   -> 11037 vols référencés valides.

[2/3] Analyse de la cible Fuel (data/raw/fuel_train.parquet)...
   -> Lignes totales : 131530
   -> Doublons exacts : 0 (Doivent être supprimés)
   -> Segments avec fuel <= 0 : 0
   -> Hypothèse Capteurs : 33.3% sont multiples de 10kg
   -> Hypothèse 'Block Fuel' : 31.7% sont multiples de 100kg

[3/3] Analyse Physique des fichiers PRC...
   -> Scan de prc801823343.parquet...
   -> Scan de prc786093351.parquet...
   -> Scan de prc800130830.parquet...
   -> Scan de prc773437294.parquet...
   -> Scan de prc775039439.parquet...
   -> Scan de prc787957716.parquet...
   -> Scan de prc779978315.parquet...
   -> Scan de prc788617196.parquet...
   -> Scan de prc782027306.parquet...
   -> Scan de prc794464264.parquet...
   -> Scan de prc786842846.parquet...
   -> Scan de prc799607832.parquet...
   -> Scan de prc790088962.parquet...
 

In [5]:
import pandas as pd
import numpy as np
import glob
import os

def analyze_column_density(path_prc_folder):
    print(">>> ANALYSE DE DENSITÉ DES DONNÉES DE VOL (PRC) <<<\n")
    
    prc_files = glob.glob(os.path.join(path_prc_folder, "prc*.parquet"))
    
    # Accumulateurs pour les statistiques globales
    total_rows = 0
    stats = {
        'TAS': {'nan': 0, 'zeros': 0},
        'mach': {'nan': 0, 'zeros': 0},
        'CAS': {'nan': 0, 'zeros': 0},
        'groundspeed': {'nan': 0, 'zeros': 0},
        'vertical_rate': {'nan': 0, 'zeros': 0}, # 0 est possible ici (palier)
        'track': {'nan': 0, 'zeros': 0}
    }
    
    for i, f in enumerate(prc_files):
        print(f"   Scan du fichier {i+1}/{len(prc_files)}...")
        df = pd.read_parquet(f)
        n = len(df)
        total_rows += n
        
        for col in stats.keys():
            if col in df.columns:
                # Compter les NaNs
                n_nan = df[col].isna().sum()
                stats[col]['nan'] += n_nan
                
                # Compter les Zéros (sauf si c'est NaN)
                # Note: Pour vertical_rate, 0 est normal. Pour TAS/Mach en vol, c'est une erreur.
                # On filtre aussi les altitudes basses pour ne pas compter le parking comme une erreur
                flight_phase = df[df['altitude'] > 1000] # On regarde seulement en vol > 1000ft
                if not flight_phase.empty:
                     n_zeros = (flight_phase[col] <= 0).sum()
                     stats[col]['zeros'] += n_zeros
            else:
                stats[col]['nan'] += n # La colonne n'existe même pas
    
    print(f"\n--- RÉSULTATS SUR {total_rows:,} POINTS ---")
    print(f"{'COLONNE':<15} | {'MANQUANTS (NaN)':<15} | {'INVALIDES (<=0 en vol)':<20} | {'DENSITÉ UTILE'}")
    print("-" * 75)
    
    for col, data in stats.items():
        pct_nan = (data['nan'] / total_rows) * 100
        # Le % de zéros est calculé approximativement sur le total (majoration du risque)
        pct_zero = (data['zeros'] / total_rows) * 100 
        useful = 100 - pct_nan - pct_zero
        
        print(f"{col:<15} | {pct_nan:6.2f}%          | {pct_zero:6.2f}%               | {useful:6.2f}%")

    print("\n>>> RECOMMANDATION STRATÉGIQUE <<<")
    if stats['TAS']['nan'] / total_rows > 0.9:
        print("CONFIRMÉ : TAS est inutilisable brute. Fallback sur Mach ou Groundspeed requis.")
    if stats['mach']['nan'] / total_rows < 0.5:
        print("OPPORTUNITÉ : Le Mach est disponible. Utiliser Mach pour reconstruire la TAS.")
    else:
        print("ATTENTION : Ni TAS ni Mach. Seul le Groundspeed est fiable.")

# Lancer l'analyse
analyze_column_density('data/raw/flights_train') # Remplacez '.' par votre dossier data

>>> ANALYSE DE DENSITÉ DES DONNÉES DE VOL (PRC) <<<

   Scan du fichier 1/11037...
   Scan du fichier 2/11037...
   Scan du fichier 3/11037...
   Scan du fichier 4/11037...
   Scan du fichier 5/11037...
   Scan du fichier 6/11037...
   Scan du fichier 7/11037...
   Scan du fichier 8/11037...
   Scan du fichier 9/11037...
   Scan du fichier 10/11037...
   Scan du fichier 11/11037...
   Scan du fichier 12/11037...
   Scan du fichier 13/11037...
   Scan du fichier 14/11037...
   Scan du fichier 15/11037...
   Scan du fichier 16/11037...
   Scan du fichier 17/11037...
   Scan du fichier 18/11037...
   Scan du fichier 19/11037...
   Scan du fichier 20/11037...
   Scan du fichier 21/11037...
   Scan du fichier 22/11037...
   Scan du fichier 23/11037...
   Scan du fichier 24/11037...
   Scan du fichier 25/11037...
   Scan du fichier 26/11037...
   Scan du fichier 27/11037...
   Scan du fichier 28/11037...
   Scan du fichier 29/11037...
   Scan du fichier 30/11037...
   Scan du fichier 31/1103